In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO



In [5]:
q = """
SELECT to_date(a.created_at, 'YYYY-mm-dd') AS ad_set_create_date,
       a.id AS ad_set_id,
       count(DISTINCT b.pid) AS product_cnt
FROM service1_quicket.ad_set a
JOIN service1_quicket.ad_set_product b ON a.id = b.ad_set_id
WHERE a.status NOT IN (2, 6)
  AND a.created_at BETWEEN '2021-02-01 00:00:00' AND '2021-02-07 23:59:59'
GROUP BY 1, 2"""
ad = pd.read_sql(q, con=bun_dw)

In [13]:
one = ad[ad['product_cnt'] == 1].groupby(['ad_set_create_date'], as_index=False)['ad_set_id'].count()
n = ad[ad['product_cnt'] > 1].groupby(['ad_set_create_date'], as_index=False)['ad_set_id'].count()

In [14]:
one.columns = ['ad_set_create_date', 'one_ad_set_cnt']
n.columns = ['ad_set_create_date', 'n_ad_set_cnt']

In [19]:
post = pd.merge(one, n, on = ['ad_set_create_date'])

In [39]:
q = """
SELECT to_date(a.created_at, 'YYYY-mm-dd') AS ad_set_create_date,
       a.id AS ad_set_id,
       count(DISTINCT b.pid) AS product_cnt
FROM service1_quicket.ad_set a
JOIN service1_quicket.ad_set_product b ON a.id = b.ad_set_id
WHERE a.status NOT IN (2, 6)
  AND a.created_at BETWEEN '2021-01-25 00:00:00' AND '2021-02-07 23:59:59'
GROUP BY 1, 2"""
pre_ad = pd.read_sql(q, con=bun_dw)

In [40]:
pre_ad['ad_set_create_date'] = pre_ad['ad_set_create_date'].astype(str)
pre_ad['ad_set_create_date'].unique()

array(['2021-01-25', '2021-01-26', '2021-01-27', '2021-01-28',
       '2021-01-29', '2021-01-30', '2021-01-31', '2021-02-01',
       '2021-02-02', '2021-02-03', '2021-02-04', '2021-02-05',
       '2021-02-06', '2021-02-07'], dtype=object)

In [41]:
pre_ad[pre_ad['ad_set_create_date'] == '2021-02-02'].groupby('product_cnt', as_index=False)['ad_set_id'].count().to_csv('ad_product_cnt0202.csv', index=False)


In [17]:
pre_one = pre_ad[pre_ad['product_cnt'] == 1].groupby(['ad_set_create_date'], as_index=False)['ad_set_id'].count()
pre_n = pre_ad[pre_ad['product_cnt'] > 1].groupby(['ad_set_create_date'], as_index=False)['ad_set_id'].count()

In [18]:
pre_one.columns = ['ad_set_create_date', 'one_ad_set_cnt']
pre_n.columns = ['ad_set_create_date', 'n_ad_set_cnt']

In [20]:
pre = pd.merge(pre_one, pre_n, on = ['ad_set_create_date'])

In [31]:
pre_ad.groupby('product_cnt', as_index=False)['ad_set_id'].count().to_csv('ad_product_cnt.csv', index=False)

In [26]:
ad_stat = pd.concat([pre, post])
ad_stat['prop'] = ad_stat['one_ad_set_cnt']/ad_stat['n_ad_set_cnt']
ad_stat.to_csv('ad_stat.csv', index=False)

In [27]:
ad_stat['prop'].mean()

3.7912121196135566

In [38]:
ad_stat

,ad_set_create_date,one_ad_set_cnt,n_ad_set_cnt,prop
0,2021-01-25,490,145,3.379310
1,2021-01-26,470,118,3.983051
2,2021-01-27,369,116,3.181034
3,2021-01-28,457,99,4.616162
4,2021-01-29,436,122,3.573770
5,2021-01-30,353,103,3.427184
6,2021-01-31,362,110,3.290909
0,2021-02-01,774,235,3.293617
1,2021-02-02,494,132,3.742424
2,2021-02-03,441,113,3.902655
